## Visualizaciones interactivas con Plotly

Plotly es una librería de código abierto que realiza visualizaciones interactivas, contiene una gran variedad de funciones para realizar y nos va a servir como herramienta principal para el desarrollo de la aplicación en Dash. Este framework también puede ser utilizado en otros lenguajes de programación como R y Java.

Enlace a la documentación de plotly: https://github.com/plotly/plotly.py/tree/master/packages/python/plotly/plotly/graph_objs

Enlace a la documentación de plotly express: https://github.com/plotly/plotly.py/tree/master/packages/python/plotly/plotly/express

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import statsmodels.api as sm

## Gráfico de líneas

Los gráficos de lineas se utilizan para observar una evolución de la variable dependiente (eje Y) a medida que la variable independiente (eje X) aumenta.



In [2]:
passengers = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv")
passengers

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [3]:
# Forma 1 para crear un gráfico de plotly

# Creo un objeto data que va a ser una lista que contiene las trazas del gráfico
data = [
    go.Scatter(
        x = passengers["Month"],
        y = passengers["Passengers"],
        mode = "lines",
        line = dict(color = "steelblue", dash = 'dash'),
        name = "Pasajeros"
    )
]

# Creo un objeto layout para definir el diseño del gráfico

layout = go.Layout(title = "Pasajeros por mes", xaxis_title = "Año y mes", yaxis_title = "Pasajeros")

# Unificamos los dos objetos dentro de la figura de plotly

fig = go.Figure(data = data, layout = layout)

# Mostramos la figura
fig.show()

In [4]:
# Forma 2 para crear un gráfico en plotly

# Creo el objeto vacío
fig = go.Figure()

# Agrego las trazas necesarias
fig.add_trace(
    go.Scatter(
        x = passengers["Month"],
        y = passengers["Passengers"],
        mode = "lines",
        line = dict(color = "steelblue", dash = 'dash'),
        name = "Pasajeros"
    )
)

# Actualizo el diseño
fig.update_layout(title = "Pasajeros por mes", xaxis_title = "Año y mes", yaxis_title = "Pasajeros")

# Muestro la figura
fig.show()

In [5]:
# Forma 3, usando Plotly Express

fig = px.line(passengers, x="Month", y="Passengers", labels={'Month':'Año y mes', 'Passengers':'Pasajeros'},
              title = "Pasajeros al mes")
fig.update_traces({"line":{"color":"steelblue", 'dash':'dash'}})
fig.show()

## Gráfico de puntos

Los gráficos de puntos (ScatterPlots) se usan principalmente para contrastar correlaciones entre dos variables. También son el objetivo de alguna visualización tras una reducción de dimensión de los datos con TSNE o PCA.



In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/rashida048/Datasets/master/StudentsPerformance.csv")
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [7]:
# Existe alguna correlación entre las notas de matemáticas y escritura

fig = px.scatter(df, x="math score", y="writing score", color="race/ethnicity", title = "Comparación de notas",
                color_discrete_map={"group A": "gold", "group B": "firebrick",
                                    "group C": "black", "group D": "mediumseagreen",
                                    "group E": "darkorange"},
                trendline="ols")
fig.show()

In [8]:
# Forma sin plotly express

colores = {"group A": "gold", "group B": "firebrick",
           "group C": "black", "group D": "mediumseagreen",
           "group E": "darkorange"}

fig = go.Figure()

for key in colores.keys():

    # Filtro por cada categoría
    aux = df[df["race/ethnicity"] == key].copy()

    # Creo la nube de puntos
    fig.add_trace(
        go.Scatter(
            x = aux["math score"],
            y = aux["writing score"],
            name = key,
            mode = "markers",
            marker_color = colores[key],
            text = aux["race/ethnicity"],
            hovertemplate='Math Score: %{x} <br>Writing Score: %{y} <br>Race/Ethnicity: %{text}'
        )
    )

    # Agrego la recta de regresión
    fit_results = sm.OLS(aux["writing score"].values, sm.add_constant(aux["math score"].values), missing="drop").fit()

    reg_x = np.array([aux["math score"].min(), aux["math score"].max()])
    reg_y = np.array([fit_results.predict().min(), fit_results.predict().max()])

    # Agregar la información de la recta
    hover_header = "<b>OLS trendline</b><br>"
    hover_header += "%s = %g * %s + %g<br>" % (
        "writing score",
        fit_results.params[1],
        "math score",
        fit_results.params[0],
    )
    hover_header += (
        "R<sup>2</sup>=%f<br><br>" % fit_results.rsquared
    )

    hover_header += 'Math Score: %{x} <br>Writing Score: %{y} <br>Race/Ethnicity: %{text}'

    # Construir la figura
    fig.add_trace(
        go.Scatter(
            x = reg_x,
            y = reg_y,
            name = "Trend " + key,
            mode = "lines",
            line = dict(color = colores[key]),
            showlegend = False,
            text = [key,key],
            hovertemplate = hover_header
        )
    )

fig.update_layout(title = "Comparación de notas", xaxis_title = "math score", yaxis_title = "writing score")

fig.show()

En este segundo caso, para dibujar la recta de regresión tendríamos que calcular la recta y agregarle la información mientras que en plotly express se hace todo con un parámetro.

## Gráficos de barras

Los gráficos de barras se usan principalmente para contar apariciones o categorías dentro de un dataset.



In [9]:
# Cómo es la distribución de la educacion de los padres los padres?
level_count = df["parental level of education"].value_counts()

data = [
    go.Bar(
        x = level_count.index,
        y = level_count,
        name = "Parental level",
        marker_color = ["red","gold","mediumseagreen","lightblue","indigo",'black'],
        width= np.repeat(0.65,len(level_count)) # Cambiar la anchura de cada barra
    )
]

layout = go.Layout(title = "Distribución de los estudios de los padres", xaxis_title = "Nivel de estudios",
                   yaxis_title = "Número de padres",
                  )

fig = go.Figure(data = data, layout = layout)

fig.show()

In [10]:
# Esto es una serie

level_count


some college          226
associate's degree    222
high school           196
some high school      179
bachelor's degree     118
master's degree        59
Name: parental level of education, dtype: int64

In [11]:
# Esto es un dataframe
pd.DataFrame(level_count).reset_index().rename(columns = {"index": "level", "parental level of education": "count"})

,level,count
0,some college,226
1,associate's degree,222
2,high school,196
3,some high school,179
4,bachelor's degree,118
5,master's degree,59


In [12]:
level_df = pd.DataFrame(level_count).reset_index().rename(columns = {"index": "level", "parental level of education": "count"})

fig = px.bar(level_df, x = "level", y = "count", title = "Distribución de los estudios de los padres")
fig.update_traces(
    marker_color = ["red","gold","mediumseagreen","lightblue","indigo",'black'],
    width= np.repeat(0.65,len(level_count))
)
fig.show()


In [13]:
# Mismo gráfico pero en horizontal

level_count.sort_values(ascending = True,inplace = True) # Reordeno para que me quede de mayor a menor el gráfico

colors =  ["red","gold","mediumseagreen","lightblue","indigo",'black']

colors.reverse()

data = [
    go.Bar(
        y = level_count.index,
        x = level_count,
        name = "Parental level",
        marker_color = colors,
        width= np.repeat(0.65,len(level_count)), # Cambiar la anchura de cada barra,
        orientation = 'h'
    )
]

layout = go.Layout(title = "Distribución de los estudios de los padres", yaxis_title = "Nivel de estudios",
                   xaxis_title = "Número de padres",
                  )

fig = go.Figure(data = data, layout = layout)

fig.show()

In [14]:
# Los grupos étnicos tienen la misma distribución de nivel de estudios?
level_count = pd.DataFrame(df.groupby("race/ethnicity")["parental level of education"].value_counts()).rename(columns = {"parental level of education": "count"}).reset_index()
level_count

,race/ethnicity,parental level of education,count
0,group A,some high school,24
1,group A,high school,18
2,group A,some college,18
3,group A,associate's degree,14
4,group A,bachelor's degree,12
5,group A,master's degree,3
6,group B,high school,48
7,group B,associate's degree,41
8,group B,some high school,38
9,group B,some college,37


In [15]:
colors = {
    "some high school": "lightblue",
    "high school": "mediumseagreen",
    'some college': "red",
    "associate's degree": "gold",
    "bachelor's degree": "indigo",
    "master's degree": "black"
}

fig = go.Figure()

for key in colors.keys():
    aux = level_count[level_count["parental level of education"] == key]
    fig.add_trace(
        go.Bar(
            x = aux["race/ethnicity"],
            y = aux["count"],
            name = key,
            marker_color = colors[key],
            width= np.repeat(0.65,len(level_count))
        )
    )

fig.update_layout(title = "Distribución de nivel de educación agrupada por raza o grupo étnico, en números absolutos",
                  xaxis_title = "Grupo étnico", yaxis_title = "Número absoluto de padres",
                  barmode='stack')

fig.show()

In [16]:
fig = go.Figure()

group_count = level_count.groupby(["race/ethnicity"])["count"].sum()

for key in colors.keys():
    aux = level_count[level_count["parental level of education"] == key]
    fig.add_trace(
        go.Bar(
            x = aux["race/ethnicity"],
            y = 100*aux["count"]/group_count.values,
            name = key,
            marker_color = colors[key],
            width= np.repeat(0.65,len(level_count))
        )
    )

fig.update_layout(title = "Distribución de nivel de educación agrupada por raza o grupo étnico, en relativo",
                  xaxis_title = "Grupo étnico", yaxis_title = "Número relativo de padres (%)",
                  barmode='stack')

fig.show()

In [17]:
# Para hacerlo con Plotly Express necesito otro formato

table = df.pivot_table(index = "race/ethnicity", columns = "parental level of education", values = 'math score',
                       aggfunc = 'count')
#table.reset_index(inplace = True)
table

parental level of education,associate's degree,bachelor's degree,high school,master's degree,some college,some high school
race/ethnicity,,,,,,
group A,14,12,18,3,18,24
group B,41,20,48,6,37,38
group C,78,40,64,19,69,49
group D,50,28,44,23,67,50
group E,39,18,22,8,35,18


In [18]:
fig = px.bar(table, x=table.index, y=table.columns, title="Distribución de nivel de educación agrupada por raza o grupo étnico, en números absolutos")
fig.show()

In [19]:
table = 100*(table.T/table.sum(axis = 1)).T

fig = px.bar(table, x=table.index, y=table.columns, title="Distribución de nivel de educación agrupada por raza o grupo étnico, en números relativos")
fig.update_layout(xaxis_title = "Grupo étnico", yaxis_title = "Número relativo de padres (%)")
fig.show()

Enlace a la documentación: https://plotly.com/python/bar-charts/

## Pie Charts

Los pie charts también nos sirve para ver la distribución de variables categóricas.



In [20]:
level_count = pd.DataFrame(df["race/ethnicity"].value_counts()).reset_index().rename(columns = {"index":"Race/Ethnicity", "race/ethnicity": "count"})
level_count = level_count.sort_values("Race/Ethnicity").reset_index(drop = True)

fig = px.pie(level_count, values="count", names="Race/Ethnicity", color = "Race/Ethnicity",
            color_discrete_map = {
                "group A": "lightblue",
                "group B": "mediumseagreen",
                "group C": "gold",
                "group D": "darkorange",
                "group E": "indigo",

            })
fig.show()

In [21]:
data = [
    go.Pie(
        labels=level_count["Race/Ethnicity"],
        values=level_count["count"],
        textinfo='label+percent',
        insidetextorientation='radial',
        marker_colors = ["lightblue", "mediumseagreen", "gold", "darkorange", "indigo"],
        pull = [0,0,0.1,0,0],
        rotation = -55,
        sort = False
    )
]

layout = go.Layout(title = "Representación de cada raza o grupo étnico dentro del dataset")

fig = go.Figure(data = data, layout = layout)

fig.show()


In [22]:
# Tambien se puede realizar como un gráfico de donut

data = [
    go.Pie(
        labels=level_count["Race/Ethnicity"],
        values=level_count["count"],
        textinfo='label+percent',
        insidetextorientation='radial',
        marker_colors = ["lightblue", "mediumseagreen", "gold", "darkorange", "indigo"],
        hole = 0.5,
        direction = "clockwise",
        sort = False
    )
]

layout = go.Layout(title = "Representación de cada raza o grupo étnico dentro del dataset")

fig = go.Figure(data = data, layout = layout)

fig.show()


## Histogramas

Los histogramas son una herramienta simple y útil de visualizar la distribución de los datos a partir de sus frecuencias.



In [23]:
# Queremos conocer la distribución de las notas de lectura

fig = px.histogram(df, x="reading score",labels = {"reading score": "Notas en lectura"},
                   nbins = 30) # Si agregamos histnorm='probability density' normalizamos el resultado

fig.update_traces(marker_color = "darkorange")
# Agregar espacios entre barras
# fig.update_layout(bargap = 0.1)
fig.show()

In [24]:
# Hay alguna asignatura que sea más sencilla que el resto?

data = [
    go.Histogram(
        x = df["math score"],
        marker_color = "firebrick",
        xbins=dict(
            start= 0,
            end= 100,
            size=5
        ),
        opacity=0.6,
        name = "Puntuación Matemáticas"
    ),
    go.Histogram(
        x = df["reading score"],
        marker_color = "mediumseagreen",
        xbins=dict(
            start= 0,
            end= 100,
            size=5
        ),
        opacity=0.6,
        name = "Puntuación Lectura"
    ),
    go.Histogram(
        x = df["writing score"],
        marker_color = "steelblue",
        xbins=dict(
            start= 0,
            end= 100,
            size=5
        ),
        opacity=0.6,
        name = "Puntuación Escritura"
    )
]

layout = go.Layout(title = "Distribución de las notas obtenidas", xaxis_title = "Puntuación", yaxis_title = "Frecuencia",
                   barmode = "overlay", bargap = 0.1)

fig = go.Figure(data = data, layout = layout)

fig.show()

In [25]:
# Ha servido el test de preparación para mejorar las notas de matemáticas

fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x = df[df["test preparation course"] == "none"]["math score"],
        marker_color = "blue",
        xbins=dict(
            start= 0,
            end= 100,
            size=5
        ),
        opacity=0.7,
        name = "Sin test de preparación"
    )
)
fig.add_trace(
    go.Histogram(
        x = df[df["test preparation course"] == "completed"]["math score"],
        marker_color = "firebrick",
        xbins=dict(
            start= 0,
            end= 100,
            size=5
        ),
        opacity=0.7,
        name = "Con test de preparación"
    )
)

fig.update_layout(title = "Distribución de las notas obtenidas", xaxis_title = "Puntuación", yaxis_title = "Frecuencia",
                   barmode = "overlay", bargap = 0.1)

fig.show()

Los boxplots, al igual que los histogramas, visualizan la distribución de los datos a partir de estadísticos simples (cuantiles, medianas, media si se añade)



In [27]:
fig = px.box(df, y="writing score", points="all")
fig.update_traces(marker_color = "steelblue")
fig.show()

In [28]:
data = [
    go.Box(
        y = df["math score"],
        marker_color = "firebrick",
        name = "Puntuación Matemáticas",
        boxpoints='all',
        boxmean=True
    ),
    go.Box(
        y = df["reading score"],
        marker_color = "mediumseagreen",
        name = "Puntuación Lectura",
        boxpoints='all',
        boxmean=True
    ),
    go.Box(
        y = df["writing score"],
        marker_color = "steelblue",
        name = "Puntuación Escritura",
        boxpoints='all',
        boxmean=True
    )
]

layout = go.Layout(title = "Distribución de las notas obtenidas", yaxis_title = "Puntuación")

fig = go.Figure(data = data, layout = layout)

fig.show()

In [29]:
# Apreciamos diferencias en las notas de lectura si se realiza el test previo?

fig = go.Figure()

fig.add_trace(
    go.Box(
        x = df[df["test preparation course"] == "none"]["reading score"],
        marker_color = "gold",
        name = "Sin test de preparación",
        boxpoints='all',
        boxmean=True
    )
)
fig.add_trace(
    go.Box(
        x = df[df["test preparation course"] == "completed"]["reading score"],
        marker_color = "mediumseagreen",
        name = "Con test de preparación",
        boxpoints='all',
        boxmean=True
    )
)

fig.update_layout(title = "Distribución de las notas obtenidas", yaxis_title = "Puntuación")

fig.show()

Enlace a la documentación: https://plotly.com/python/box-plots/

## Multiples gráficas en una sola


In [32]:
from plotly.subplots import make_subplots

# Buscamos un resumen de toda la información sobre las notas de escritura


# Creo el objeto que me permite hacer 4 gráficas al mismo tiempo
fig = make_subplots(rows = 2,
                    cols = 2,
                    subplot_titles=("Distribución de las notas", "Impacto del test previo",
                                    "Media según la formación de los padres",
                                    "Media por formación y grupo étnico"))


# primer gráfico: Histograma de valores de las notas de writing
fig.add_trace(
    go.Histogram(
        x = df["writing score"],
        name = "Notas de Escritura",
        marker_color = "steelblue",
        xbins=dict(
            start= 0,
            end= 100,
            size=5
        ),
        showlegend = False
    ),
    row = 1,
    col = 1
)

# Segundo gráfico: Como ha afectado los test de preparación

fig.add_trace(
    go.Box(
        y = df[df["test preparation course"] == "none"]["writing score"],
        marker_color = "gold",
        name = "Sin test",
        boxpoints='all',
        boxmean=True,
        showlegend = False
    ),
    row = 1,
    col = 2
)
fig.add_trace(
    go.Box(
        y = df[df["test preparation course"] == "completed"]["writing score"],
        marker_color = "mediumseagreen",
        name = "Con test",
        boxpoints='all',
        boxmean=True,
        showlegend = False
    ),
    row = 1,
    col = 2
)

# Tercer gráfico: Cual es la media de las notas según el nivel de estudios de los padres
groups = pd.DataFrame(df.groupby("parental level of education")["writing score"].mean()).sort_values("writing score")

fig.add_trace(
    go.Bar(
        y = groups.index,
        x = groups["writing score"],
        name = "Media por grupo",
        showlegend = False,
        marker_color = ["lightblue", "mediumseagreen", "gold", "darkorange", "indigo"],
        orientation = "h"
    ),
    row = 2,
    col = 1
)

# Cuarto gráfico: Como influye la raza del alumno/a y el nivel de educación de los padres en la media
groups = pd.DataFrame(df.groupby(["parental level of education", "race/ethnicity"])["writing score"].mean()).reset_index()

fig.add_trace(
    go.Heatmap(
        x = groups["parental level of education"],
        y = groups["race/ethnicity"],
        z = groups["writing score"],
        name = "Media por grupo y educación de los padres",
        colorscale = 'RdBu',
        showlegend = False,
        showscale = False
    ),
    row = 2,
    col = 2
)

# Modifico las dimensiones totales y el titulo global
fig.update_layout(width = 950, height = 800, title = "Información sobre la asignatura de escritura", bargap = 0.1)


fig.update_xaxes(title_text = "Puntuación", row = 1, col = 1)
fig.update_xaxes(title_text = "Puntuación media", row = 2, col = 1)

fig.update_yaxes(title_text = "Frecuencia", row = 1, col = 1)
fig.update_yaxes(title_text = "Puntuación", row = 1, col = 2)

fig.show()

Enlace a subplots: https://plotly.com/python/subplots/

Enlace a la documentacion de heatmaps: https://plotly.com/python/heatmaps/